In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seed for reproducibility
torch.manual_seed(64)

# Path for saving models and results
save_path = 'Training_results/Harvey_5foldcv_BERT_Multihead_512/'
os.makedirs(save_path, exist_ok=True)

# Load BERT tokenizer and model from HuggingFace's transformers library
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

def haversine(lat1, lon1, lat2, lon2):
    R = 3959
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

# Define dataset class to handle tokenization and data loading
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenizing the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float32)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(BERTDeepMultiHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)

        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

def kfoldcv(grouped_tweets, grouped_targets, target_list=[], max_len=256, batch_size=32, num_epochs=3):
    saved_models = []
    target_list = get_target_list(target_list)

    tweet_dict = {int(name): group['text'] for name, group in grouped_tweets}
    target_dict = {int(name): group[target_list] for name, group in grouped_targets}

    intersecting_zips = list(set(target_dict.keys()) & set(tweet_dict.keys()))
    paired_data = {
        name: [target_dict[name].sum().apply(lambda x: 1 if x > 0 else 0), tweet_dict[name]] for name in intersecting_zips
    }

    texts = ['\n'.join(v[1].to_list()) for v in paired_data.values()]
    labels_ = [v[0] for v in paired_data.values()]

    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_results = []

    # print(len(texts), len(labels))

    for fold, (train_idx, test_idx) in enumerate(kfold.split(texts, labels_)):
        # print(f"Train indices: {train_idx}")
        # print(f"Test indices: {test_idx}")
        print(f"Training fold {fold + 1}...")

        X_train, X_test = [texts[i] for i in train_idx], [texts[i] for i in test_idx]
        y_train, y_test = [labels_[i] for i in train_idx], [labels_[i] for i in test_idx]

        train_dataset = CustomDataset(X_train, y_train, tokenizer, max_len)
        test_dataset = CustomDataset(X_test, y_test, tokenizer, max_len)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-5)
        criterion = [nn.BCELoss() for _ in range(len(target_list))]

        best_f1 = 0.0  # Track the best validation F1 score

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
            for batch in progress_bar:
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask)
                loss = sum(c(outputs[:, i], labels[:, i]) for i, c in enumerate(criterion))
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix(loss=loss.item())

            model.eval()
            val_loss = 0.0
            all_preds, all_labels = [], []
            progress_bar = tqdm(test_loader, desc=f"Epoch {epoch+1} Validation")
            with torch.no_grad():
                for batch in progress_bar:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)

                    outputs = model(input_ids, attention_mask)
                    loss = sum(c(outputs[:, i], labels[:, i]) for i, c in enumerate(criterion))
                    val_loss += loss.item()

                    all_preds.append(outputs.cpu().numpy())
                    all_labels.append(labels.cpu().numpy())
                    progress_bar.set_postfix(loss=loss.item())

            all_preds = np.vstack(all_preds)
            all_labels = np.vstack(all_labels)
            f1 = [f1_score(all_labels[:, i], all_preds[:, i] > 0.5) for i in range(len(target_list))]
            accuracy = [accuracy_score(all_labels[:, i], all_preds[:, i] > 0.5) for i in range(len(target_list))]
            precision = [precision_score(all_labels[:, i], all_preds[:, i] > 0.5, zero_division=0) for i in range(len(target_list))]
            recall = [recall_score(all_labels[:, i], all_preds[:, i] > 0.5) for i in range(len(target_list))]
            mean_f1 = np.mean(f1)
            mean_accuracy = np.mean(accuracy)
            mean_precision = np.mean(precision)
            mean_recall = np.mean(recall)

            print(f"Epoch {epoch+1}: Train Loss={train_loss/len(train_loader):.4f}, Val Loss={val_loss/len(test_loader):.4f}")
            print(f"Mean F1 Score: {mean_f1:.4f}, Mean Accuracy: {mean_accuracy:.4f}, Mean Precision: {mean_precision:.4f}, Mean Recall: {mean_recall:.4f}")

            # Save the best model based on validation F1 score
            if mean_f1 > best_f1:
                best_f1 = mean_f1
                best_model_path = os.path.join(save_path, f"best_bert_model_fold_{fold+1}.pth")
                torch.save(model.state_dict(), best_model_path)
                print(f"New best model saved for fold {fold+1} with F1={best_f1:.4f}")

        saved_models.append(best_model_path)

    return fold_results, saved_models

In [ ]:
size_threshold = 80

tweets_harvey2 = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/tweets/harvey_corrected.csv')
tweets_harvey2.rename(columns={'zipcode': 'zip_code'}, inplace=True)

bboxes_useful = tweets_harvey2.place_bbox.apply(lambda x: [[float(i.strip('()[]')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_harvey = tweets_harvey2.loc[((tweets_harvey2.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)), :]
tweets_harvey.loc[:, 'zip_code'] = tweets_harvey['zip_code'].apply(int)
tweet_grouped_harvey = tweets_harvey.groupby('zip_code')

targets_harvey = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/targets/disaster_4332.csv')
target_grouped_harvey = targets_harvey.groupby('damagedZipCode')

# After running the kfoldcv function:
fold_results, saved_models = kfoldcv(tweet_grouped_harvey, target_grouped_harvey, max_len = 512, batch_size=24, num_epochs = 100)

C:\Users\udays\AppData\Local\Temp\ipykernel_17740\1237475545.py:12: DtypeWarning: Columns (9,10,13,28,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  targets_harvey = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/targets/disaster_4332.csv')


Training fold 1...


C:\Users\udays\AppData\Local\Temp\ipykernel_17740\406069421.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 1 Validation: 100%|█████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.76it/s, loss=7.03]


Epoch 1: Train Loss=7.4118, Val Loss=7.0306
Mean F1 Score: 0.5720, Mean Accuracy: 0.6530, Mean Precision: 0.5599, Mean Recall: 0.7290
New best model saved for fold 1 with F1=0.5720


C:\Users\udays\AppData\Local\Temp\ipykernel_17740\406069421.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 2 Validation: 100%|█████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.69it/s, loss=6.77]


...

Epoch 98: Train Loss=0.1784, Val Loss=0.2792
Mean F1 Score: 0.9942, Mean Accuracy: 0.9921, Mean Precision: 0.9885, Mean Recall: 1.0000


C:\Users\udays\AppData\Local\Temp\ipykernel_17740\406069421.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 99 Validation: 100%|██████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.41it/s, loss=0.0482]


Epoch 99: Train Loss=0.1753, Val Loss=0.2762
Mean F1 Score: 0.9942, Mean Accuracy: 0.9921, Mean Precision: 0.9885, Mean Recall: 1.0000


C:\Users\udays\AppData\Local\Temp\ipykernel_17740\406069421.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 100 Validation: 100%|█████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.42it/s, loss=0.0464]

Epoch 100: Train Loss=0.1704, Val Loss=0.2743
Mean F1 Score: 0.9942, Mean Accuracy: 0.9921, Mean Precision: 0.9885, Mean Recall: 1.0000


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(64)

# Load the saved models
# save_path = 'Training_results/Harvey_5foldcv_BERT_Multihead/'
saved_model_paths = [os.path.join(save_path, f"best_bert_model_fold_{i+1}.pth") for i in range(5)]
models = []

# Load BERT tokenizer and model
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = transformers.BertModel.from_pretrained('bert-base-uncased')

# Get the target list
target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

def haversine(lat1, lon1, lat2, lon2):
    R = 3959
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2

# Get the target list if not provided
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list

# Define dataset class to handle tokenization and data loading
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenizing the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float32)
        }

class BERTDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(BERTDeepMultiHeadClassifier, self).__init__()
        self.bert = model_bert
        self.drop = nn.Dropout(0.3)

        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.bert.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)

# Load all trained models
for path in saved_model_paths:
    model = BERTDeepMultiHeadClassifier(num_targets=len(target_list)).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    models.append(model)

size_threshold = 80

tweets_harvey2 = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/tweets/harvey_corrected.csv')
tweets_harvey2.rename(columns={'zipcode': 'zip_code'}, inplace=True)

bboxes_useful = tweets_harvey2.place_bbox.apply(lambda x: [[float(i.strip('()[]')) for i in x.split(', ')][i] for i in [1,0,3,2]])
bbu_areas = bboxes_useful.apply(lambda x: get_box_area(*x))
tweets_harvey = tweets_harvey2.loc[((tweets_harvey2.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas < size_threshold)), :]
tweets_harvey.loc[:, 'zip_code'] = tweets_harvey['zip_code'].apply(int)
tweet_grouped_harvey = tweets_harvey.groupby('zip_code')

targets_harvey = pd.read_csv('D:/TAMIDS/Disaster_Impact_Estimation/targets/disaster_4332.csv')
target_grouped_harvey = targets_harvey.groupby('damagedZipCode')

tweet_dict = {int(name): group['text'] for name, group in tweet_grouped_harvey}
target_dict = {int(name): group[target_list] for name, group in target_grouped_harvey}

intersecting_zips = list(set(target_dict.keys()) & set(tweet_dict.keys()))
paired_data = {
    name: [target_dict[name].sum().apply(lambda x: 1 if x > 0 else 0), tweet_dict[name]] for name in intersecting_zips
}

# Prepare dataset for inference
texts = ['\n'.join(v[1].to_list()) for v in paired_data.values()]
labels_ = [v[0] for v in paired_data.values()]
zip_codes = list(paired_data.keys())

test_dataset = CustomDataset(texts, labels_, tokenizer, max_len=512)
test_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)

# Store predictions
all_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Making predictions", unit="batch"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        fold_preds = []
        for model in models:
            outputs = model(input_ids, attention_mask)
            fold_preds.append(outputs.cpu().numpy())

        fold_preds = np.stack(fold_preds, axis=0)  # Shape: (num_models, batch_size, num_targets)
        all_preds.append(fold_preds)

# Convert list to numpy array
all_preds = np.concatenate(all_preds, axis=1)  # Shape: (num_models, total_samples, num_targets)
mean_preds = all_preds.mean(axis=0)  # Mean prediction probabilities (Shape: total_samples, num_targets)
binary_preds = (mean_preds > 0.5).astype(int)  # Convert probabilities to binary predictions

# Convert labels to numpy array
true_labels = np.vstack(labels_)

# Compute evaluation metrics
metrics_dict = {'Target': target_list, 'Accuracy': [], 'F1_Score': [], 'Precision': [], 'Recall': []}

for i in range(len(target_list)):
    metrics_dict['Accuracy'].append(accuracy_score(true_labels[:, i], binary_preds[:, i]))
    metrics_dict['F1_Score'].append(f1_score(true_labels[:, i], binary_preds[:, i]))
    metrics_dict['Precision'].append(precision_score(true_labels[:, i], binary_preds[:, i]))
    metrics_dict['Recall'].append(recall_score(true_labels[:, i], binary_preds[:, i]))

# Compute and append average metrics
metrics_dict['Target'].append('Average')
metrics_dict['Accuracy'].append(np.mean(metrics_dict['Accuracy']))
metrics_dict['F1_Score'].append(np.mean(metrics_dict['F1_Score']))
metrics_dict['Precision'].append(np.mean(metrics_dict['Precision']))
metrics_dict['Recall'].append(np.mean(metrics_dict['Recall']))

# Convert to DataFrame and save
metrics_df = pd.DataFrame(metrics_dict)
metrics_df_path = os.path.join(save_path, "bert_metrics.xlsx")
metrics_df.to_excel(metrics_df_path, index=False)

target_list = [
    'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage',
    'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible',
    'replacementAssistanceEligible', 'personalPropertyEligible'
]

# Save mean prediction probabilities
mean_preds_df = pd.DataFrame(mean_preds, columns=target_list)
mean_preds_df.insert(0, "zip_code", zip_codes)
mean_preds_path = os.path.join(save_path, "bert_mean_predictions.xlsx")
mean_preds_df.to_excel(mean_preds_path, index=False)

# Save binary predictions
binary_preds_df = pd.DataFrame(binary_preds, columns=target_list)
binary_preds_df.insert(0, "zip_code", zip_codes)
binary_preds_path = os.path.join(save_path, "bert_binary_predictions.xlsx")
binary_preds_df.to_excel(binary_preds_path, index=False)

print(f"Metrics saved to {metrics_df_path}")
print(f"Mean predictions saved to {mean_preds_path}")
print(f"Binary predictions saved to {binary_preds_path}")

C:\Users\udays\AppData\Local\Temp\ipykernel_17740\2975026519.py:111: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path))
C:\Users\udays\App

Metrics saved to Training_results/Harvey_5foldcv_BERT_Multihead_512/bert_metrics.xlsx
Mean predictions saved to Training_results/Harvey_5foldcv_BERT_Multihead_512/bert_mean_predictions.xlsx
Binary predictions saved to Training_results/Harvey_5foldcv_BERT_Multihead_512/bert_binary_predictions.xlsx
